In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
posts_df = pd.read_csv('./neural_networks_1/posts_df.csv')

In [4]:
posts_df.shape

(262577, 8)

In [5]:
posts_df.head(3) # вижу - какая-то фигня. Попробовала разобраться с исходниками - не получилось.
# решила слепить нормальный дф из того, что есть

,time,tags,nsfw,my,images,videos,text_len,rating
0,"1569877228,""Творчество детское, Рисунок, Детск...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1569877232,""Собака, Было-Стало, {#73#}"",0,1,1,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1569877241,""Дерево, Поле, Фотография"",0,0,1,0,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


`time` - время публикации поста в формате unix timestamp

`tags` - тематические теги поста

`nsfw` - относится ли пост к тематике 18+

`my` - является ли пост оригинальным авторским

`images` - количество картинок в посте

`videos` - количество видео в посте

`text_len` - длина текста в посте

`rating` - финальный рейтинг поста (целевая переменная)

In [ ]:
"""  проверяю, есть ли все-таки нормальные строки в исходнике
Для этого создам отдельный ДФ
Имеющиеся нормальные строки солью в него.
А из рабочего ДФ их удалю
"""

In [ ]:
''' можно воспользоваться особым свойством NaN:

In [23]: np.nan == np.nan
Out[23]: False
Т.е. NaN не равен самому себе. Это даёт нам возможность воспользоваться 
удобным методом DataFrame.query():

print(df.query("tags == tags"))
выведет все строки для которых tags == tags - т.е. только те для которых tags != NaN
'''

In [6]:
print(posts_df.query("tags == tags")) # таких строк - 135

              time        tags  nsfw   my  images  videos  text_len  rating
1231    1569940607       Текст   0.0  0.0     0.0     0.0       4.0   426.0
1232    1569940611       Текст   0.0  0.0     0.0     1.0      14.0   114.0
1282    1569942149       Текст   0.0  0.0     0.0     0.0     104.0   478.0
2132    1569993017  Длиннопост   0.0  0.0    11.0     0.0    7901.0   966.0
12645   1570463316  Длиннопост   0.0  0.0     6.0     0.0    4293.0   795.0
...            ...         ...   ...  ...     ...     ...       ...     ...
231037  1580734219  Длиннопост   0.0  0.0     6.0     0.0   14197.0  1321.0
244279  1581333475       Текст   0.0  0.0     0.0     0.0     100.0     8.0
244355  1581336170       Текст   0.0  0.0     0.0     0.0     327.0   699.0
258505  1581951974       Текст   0.0  0.0     0.0     0.0     498.0   617.0
262213  1582113627       Текст   0.0  0.0     1.0     0.0     349.0   613.0

[135 rows x 8 columns]


In [7]:
correct_lines = posts_df.query("tags == tags").index # дает True (т.е. tags != NaN)- индексы  строк
correct_lines

Int64Index([  1231,   1232,   1282,   2132,  12645,  14495,  14641,  18257,
             18572,  19570,
            ...
            203316, 209708, 221012, 228576, 230762, 231037, 244279, 244355,
            258505, 262213],
           dtype='int64', length=135)

In [8]:
# создадим новый ДФ с "нормальными" строками из исходного ДФ
correct_lines_df = posts_df.iloc[correct_lines]
correct_lines_df.head(3)

,time,tags,nsfw,my,images,videos,text_len,rating
1231,1569940607,Текст,0.0,0.0,0.0,0.0,4.0,426.0
1232,1569940611,Текст,0.0,0.0,0.0,1.0,14.0,114.0
1282,1569942149,Текст,0.0,0.0,0.0,0.0,104.0,478.0


In [9]:
# удалить из исходного ДФ "нормальные строки"
#проверяю по индексу, правильная ли это строка
posts_df.iloc[correct_lines].tail(3)

,time,tags,nsfw,my,images,videos,text_len,rating
244355,1581336170,Текст,0.0,0.0,0.0,0.0,327.0,699.0
258505,1581951974,Текст,0.0,0.0,0.0,0.0,498.0,617.0
262213,1582113627,Текст,0.0,0.0,1.0,0.0,349.0,613.0


In [10]:
# удаляю из исходного ДФ "нормальные строки"
posts_df = posts_df.drop(index = correct_lines)

In [11]:
#снова проверяю по индексу, стали ли правильные строки - неправильными
posts_df.iloc[correct_lines].tail(3)

,time,tags,nsfw,my,images,videos,text_len,rating
244488,"1581341086,""Продукты питания, Кулинария, Субпр...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
258639,"1581955850,""Авито, Развод, Собака, Передержка,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
262348,"1582118047,""Чай, Лес, Зима, Видео"",0,1,0,1,0,445",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
"""ПОЕХАЛИ
Разделять строку на столбики
"""

In [12]:
# создадим новый столбец со строкой, которую надо разделить
posts_df['string_to_split'] = posts_df['time']
posts_df['time'] = 0
posts_df.head(2)

,time,tags,nsfw,my,images,videos,text_len,rating,string_to_split
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1569877228,""Творчество детское, Рисунок, Детск..."
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1569877232,""Собака, Было-Стало, {#73#}"",0,1,1,..."


In [13]:
###### убрать пробелы, убрать двойные кавычки, заменить запятую на пробел, преобразовать в list

to_list = [] # лист для нового столбца, "вся эта инфа => list"
for string in posts_df['string_to_split']:
    string = string.replace(' ','')
    string = string.replace('"','')
    string = string.replace(',',' ')
    string = string.split()
    to_list.append(string)

In [14]:
to_list[5:7]

[['1569877284',
  'Доставкаеды',
  'Плохойсервис',
  'Непраздник',
  '0',
  '1',
  '1',
  '0',
  '1260',
  '515'],
 ['1569877308',
  'Motherfucker',
  'Мультфильмы',
  'Обзор',
  'Аниме',
  'Ублюдыш',
  'Комиксы',
  'Видео',
  '0',
  '0',
  '0',
  '1',
  '0',
  '219']]

In [15]:
# создать колонку из листа листов
posts_df['string_to_list'] = to_list
posts_df = posts_df.drop("string_to_split", axis=1)
posts_df.head(3)

,time,tags,nsfw,my,images,videos,text_len,rating,string_to_list
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1569877228, Творчестводетское, Рисунок, Детск..."
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1569877232, Собака, Было-Стало, {#73#}, 0, 1,..."
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1569877241, Дерево, Поле, Фотография, 0, 0, 1..."


In [16]:
######## извлечь из листа инфу для каждой из колонок
time_to_list = []
tags_to_list = []
nsfw_to_list = []
my_to_list = []
images_to_list = []
videos_to_list = []
text_len_to_list = []
rating_to_list = []

for to_list in posts_df['string_to_list']:

    time = int(to_list[0])
    time_to_list.append(time)
    
    tags = ', '.join(to_list[1:-6])
    tags_to_list.append(tags)
    
    nsfw = int(to_list[-6])
    nsfw_to_list.append(nsfw)
    
    my = int(to_list[-5])
    my_to_list.append(my)
    
    images = int(to_list[-4])
    images_to_list.append(images)
    
    videos = int(to_list[-3])
    videos_to_list.append(videos)
    
    text_len = int(to_list[-2])
    text_len_to_list.append(text_len)
    
    rating = int(to_list[-1])
    rating_to_list.append(rating)
    

posts_df['time'] = time_to_list
posts_df['tags'] = tags_to_list
posts_df['nsfw'] = nsfw_to_list
posts_df['my'] = my_to_list
posts_df['images'] = images_to_list
posts_df['videos'] = videos_to_list
posts_df['text_len'] = text_len_to_list
posts_df['rating'] = rating_to_list
posts_df = posts_df.drop("string_to_list", axis=1)

In [17]:
posts_df.head(3)

,time,tags,nsfw,my,images,videos,text_len,rating
0,1569877228,"Творчестводетское, Рисунок, Детскийсад, Детски...",0,1,2,0,135,683
1,1569877232,"Собака, Было-Стало, {#73#}",0,1,1,0,113,480
2,1569877241,"Дерево, Поле, Фотография",0,0,1,0,0,301


In [18]:
len(posts_df)

262442

In [19]:
len(correct_lines_df)

135

In [20]:
#проверим типы данных
posts_df.dtypes

time         int64
tags        object
nsfw         int64
my           int64
images       int64
videos       int64
text_len     int64
rating       int64
dtype: object

In [21]:
correct_lines_df.dtypes

time         object
tags         object
nsfw        float64
my          float64
images      float64
videos      float64
text_len    float64
rating      float64
dtype: object

In [22]:
# приведем типы данных в соответствие
columns = ['time', 'nsfw', 'my', 'images', 'videos', 'text_len', 'rating']
columns

['time', 'nsfw', 'my', 'images', 'videos', 'text_len', 'rating']

In [23]:
correct_lines_df = correct_lines_df.astype(
    ({'time': np.int64, 
      'nsfw': np.int64, 
      'my': np.int64, 
      'images': np.int64, 
      'videos': np.int64, 
      'text_len': np.int64, 
      'rating': np.int64, })
)

In [24]:
correct_lines_df.dtypes

time         int64
tags        object
nsfw         int64
my           int64
images       int64
videos       int64
text_len     int64
rating       int64
dtype: object

In [25]:
# сконкатенировать два ДФ-ма
df = pd.concat([posts_df, correct_lines_df], ignore_index=True)
df.head(3)

,time,tags,nsfw,my,images,videos,text_len,rating
0,1569877228,"Творчестводетское, Рисунок, Детскийсад, Детски...",0,1,2,0,135,683
1,1569877232,"Собака, Было-Стало, {#73#}",0,1,1,0,113,480
2,1569877241,"Дерево, Поле, Фотография",0,0,1,0,0,301


In [26]:
len(df)

262577

In [27]:
df.dtypes

time         int64
tags        object
nsfw         int64
my           int64
images       int64
videos       int64
text_len     int64
rating       int64
dtype: object

Давайте из столбца `time` вытащим сведения о том, в какое время суток был опубликован пост. 

Но если мы будем кодировать время суток как просто количество секунд, прошедшее с определённого момента, то мы не сможем учесть тот факт, что минимальные и максимальные моменты времени достаточно близки друг к другу. Так, например, время 23:59 и 00:01 - практически одно и то же, но на нашей шкале они будут максимально далеко друг от друга. 

Для того, чтобы избежать такого эффекта, представим время как угловую координату на окружности (меняющуюся от 0 до 2pi) и закодируем его в виде двух переменных: синуса и косинуса этого угла. В таком случае близкие друг к другу моменты времени будут иметь близкие значения во входных данных.

In [28]:
# получим количество секунд, прошедшее с начала суток
df['time'] = df['time'] % (60*60*24)

# и отнормируем все значения на 2pi
df['time'] = (df['time'] / df['time'].max()) * 2*math.pi

In [29]:
# добавим колонки с синусом и косинусом нашего времени
tsin_list = []
tcos_list = []
for row in df['time']:
    tsin_list.append(math.sin(row))
    tcos_list.append(math.cos(row))

df['tsin'] = tsin_list
df['tcos'] = tcos_list

In [30]:
df = df.drop(columns=['time'])

In [31]:
df.head()

,tags,nsfw,my,images,videos,text_len,rating,tsin,tcos
0,"Творчестводетское, Рисунок, Детскийсад, Детски...",0,1,2,0,135,683,-0.705620,0.708590
1,"Собака, Было-Стало, {#73#}",0,1,1,0,113,480,-0.705414,0.708795
2,"Дерево, Поле, Фотография",0,0,1,0,0,301,-0.704950,0.709257
3,"Военторг, Изсети, Продукты, Длиннопост",0,0,4,0,0,425,-0.704537,0.709667
4,"MortalKombat11, Тизер, Текст, Геймеры, Новости...",0,0,1,0,726,617,-0.704021,0.710179


Теперь разберёмся с тегами. Их достаточно много разных.

Среди них встречаются теги вида `{#73#}` - это теги сообществ. Т.к. они достаточно редки, давайте уберём их, но добавим в наш датасет дополнительный признак - был ли пост опубликован в сообществе.

In [32]:
#сделаем из строк списки с тегами
df['tags'] = df['tags'].apply(lambda x: x.split(', '))

In [33]:
in_comm_list = []  # новый столбец в нашем датасете, "был ли пост опубликован в сообществе"

# удалим теги, начинающиеся и заканчивающиеся фигурной скобкой, 
# при этом запомнив, что пост был в сообществе
for row in df['tags']:
    is_in_comm = 0
    for tag in row:
        if tag[0] == '{' and tag[-1] == '}':
            row.remove(tag)
            is_in_comm = 1
    in_comm_list.append(is_in_comm)

df['in_comm'] = in_comm_list

Теперь давайте посмотрим, сколько вообще у нас имеется разных тегов:

In [34]:
# воспользуемся множествами, чтобы каждый тег учесть только в одном экземпляре
tags_set = set()

# добавим все имеющиеся теги в множество
for row in df['tags']:
    for tag in row:
        tags_set.add(tag)

print(f"Количество используемых тегов: {len(tags_set)}")

Количество используемых тегов: 63296


Тегов много, но многие из них используются достаточно редко и вряд ли дадут нам много информации. Поэтому давайте при помощи One-Hot Encoding закодируем 50 самых популярных тегов, а если никакие из них не встречаются в посте, укажем это в дополнительном столбце `other`.

In [35]:
# создадим словарь для подсчёта того, 
# сколько раз каждый тег встречается в нашем датасете
tags_dict = {tag: 0 for tag in tags_set}

for row in df['tags']:
    for tag in row:
        tags_dict[tag] += 1

In [36]:
# преобразуем словарь в список пар (тег, кол-во вхождений)
# и отсортируем его по убыванию числа вхождений тега
tags_list = list(tags_dict.items())
tags_list.sort(key=lambda x: x[1], reverse=True)

In [37]:
# 10 самых популярных тегов:
tags_list[:10]

[('Длиннопост', 75128),
 ('Текст', 59256),
 ('Видео', 34107),
 ('Юмор', 14957),
 ('Кот', 13512),
 ('Фотография', 10869),
 ('Мат', 8662),
 ('Скриншот', 8251),
 ('Арт', 7585),
 ('Комиксы', 6446)]

In [38]:
# 10 самых непопулярных тегов:
tags_list[-10:]

[('Чернобров', 1),
 ('т-150', 1),
 ('ФранкоДзеффирелли', 1),
 ('Розовыйжемчуг', 1),
 ('Rugo', 1),
 ('Ямочки', 1),
 ('Перевоспитание', 1),
 ('Транснациональныекомпании', 1),
 ('Вакуумнаякамера', 1),
 ('LauraBodewig', 1)]

In [39]:
# будем использовать только 50 самых популярных тегов
tags2use = 50
popular_tags = [t[0] for t in tags_list[:tags2use]]

In [40]:
# создадим в датафрейме колонку под каждый тег, 
# где отметим, встречался ли этот тег в посте
for tag in popular_tags:
    df[tag] = df['tags'].apply(lambda x: int(tag in x))

In [41]:
# в столбце `other` сделаем отметку, 
# если пост не описывается популярными тегами 
# (т.е. в столбцах с тегами стоят одни нули)
df['other'] = df[popular_tags].sum(axis=1)
df['other'] = df['other'].apply(lambda x: int(x == 0))

In [42]:
df.head()

,tags,nsfw,my,images,videos,text_len,rating,tsin,tcos,in_comm,...,Своимируками,MyLittlePony,Музыка,Мошенничество,Отношения,Стихи,Зима,СССР,Еда,other
0,"[Творчестводетское, Рисунок, Детскийсад, Детск...",0,1,2,0,135,683,-0.705620,0.708590,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Собака, Было-Стало]",0,1,1,0,113,480,-0.705414,0.708795,1,...,0,0,0,0,0,0,0,0,0,0
2,"[Дерево, Поле, Фотография]",0,0,1,0,0,301,-0.704950,0.709257,0,...,0,0,0,0,0,0,0,0,0,0
3,"[Военторг, Изсети, Продукты, Длиннопост]",0,0,4,0,0,425,-0.704537,0.709667,0,...,0,0,0,0,0,0,0,0,0,0
4,"[MortalKombat11, Тизер, Текст, Геймеры, Новост...",0,0,1,0,726,617,-0.704021,0.710179,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
df[popular_tags + ['other']].mean()

Длиннопост                0.286118
Текст                     0.225671
Видео                     0.129893
Юмор                      0.056962
Кот                       0.051459
Фотография                0.041394
Мат                       0.032272
Скриншот                  0.031423
Арт                       0.028887
Комиксы                   0.024549
Аниме                     0.022454
История                   0.020272
AnimeArt                  0.020268
Собака                    0.019362
Безрейтинга               0.019122
КомментариинаПикабу       0.018303
Политика                  0.017599
НовыйГод                  0.017515
Дети                      0.017092
Рисунок                   0.016806
Картинкастекстом          0.016209
Новости                   0.016163
Девушки                   0.015782
Помощь                    0.015100
Россия                    0.014929
Котомафия                 0.014918
Работа                    0.014571
Гифка                     0.014464
Игры                

In [44]:
df = df.drop(columns=['tags'])

In [ ]:
df.to_csv('prepared_posts_df.csv', index = False)